# 6.10 Homework
The goal of this homework is to create a tree-based regression model for prediction apartment prices (column 'price').

In this homework we'll again use the [New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data) dataset - the same one we used in homework 2 and 3.

Let's load the data:

In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score, accuracy_score, mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.tree import export_text, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [3]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]

df = pd.read_csv('dataset/AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

- Apply the log tranform to price
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the train_test_split function and set the random_state parameter to 1

In [4]:
df.head()

,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Brooklyn,40.64749,-73.97237,Private room,149,1,9,0.21,6,365
1,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,0.38,2,355
2,Manhattan,40.80902,-73.94190,Private room,150,3,0,0.00,1,365
3,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,4.64,1,194
4,Manhattan,40.79851,-73.94399,Entire home/apt,80,10,9,0.10,1,0


In [8]:
df["price"] = np.log1p(df["price"])

X = df.drop(["price"], axis=1)
y = df["price"]
all_X_train, X_test, all_y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(all_X_train, all_y_train, test_size=0.2, random_state=1)
print(len(X_train)/len(X), len(X_val)/len(X), len(X_test)/len(X))

0.6399836384088352 0.16001636159116475 0.2


Now, use DictVectorizer to turn train and validation into matrices:

In [12]:
dv = DictVectorizer(sparse=False)

train_dict = X_train.to_dict(orient="records")
X_train = dv.fit_transform(train_dict)

val_dict = X_val.to_dict(orient="records")
X_val = dv.transform(val_dict)

# Question 1

Let's train a decision tree regressor to predict the price variable.

- Train a model with max_depth=1

In [19]:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)
print(export_text(dt, feature_names=dv.get_feature_names()))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [1.66]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [1.81]



Which feature is used for splitting the data?

- **room_type**
- neighbourhood_group
- number_of_reviews
- reviews_per_month

# Question 2

Train a random forest model with these parameters:

- n_estimators=10
- random_state=1
- n_jobs=-1 (optional - to make training faster)

In [23]:
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
print(round(math.sqrt(mean_squared_error(y_pred, y_val)), 3))

0.082


What's the RMSE of this model on validation?

- **0.059**
- 0.259
- 0.459
- 0.659

# Question 3

Now let's experiment with the n_estimators parameter

- Try different values of this parameter from 10 to 200 with step 10
- Set random_state to 1
- Evaluate the model on the validation dataset

In [24]:
for n in range(10, 200, 10):
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    print(n, round(math.sqrt(mean_squared_error(y_pred, y_val)), 3))

10 0.082
20 0.08
30 0.08
40 0.079
50 0.079
60 0.079
70 0.079
80 0.079
90 0.079
100 0.079
110 0.079
120 0.079
130 0.079
140 0.079
150 0.079
160 0.079
170 0.079
180 0.079
190 0.079


After which value of n_estimators does RMSE stop improving?

- 10
- **50**
- 70
- 120

# Question 4

Let's select the best max_depth:

- Try different values of max_depth: [10, 15, 20, 25]
- For each of these values, try different values of n_estimators from 10 till 200 (with step 10)
- Fix the random seed: random_state=1

In [26]:
for d in [10, 15, 20, 25]:
    print(f"Depth {d}")
    for n in range(10, 200, 10):
        rf = RandomForestRegressor(n_estimators=n, max_depth=d, random_state=1, n_jobs=-1)
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_val)
        print(n, round(math.sqrt(mean_squared_error(y_pred, y_val)), 3))

Depth 10
10 0.08
20 0.079
30 0.079
40 0.079
50 0.079
60 0.079
70 0.079
80 0.079
90 0.079
100 0.079
110 0.079
120 0.079
130 0.079
140 0.079
150 0.079
160 0.079
170 0.079
180 0.079
190 0.079
Depth 15
10 0.081
20 0.079
30 0.079
40 0.078
50 0.078
60 0.078
70 0.078
80 0.078
90 0.078
100 0.078
110 0.078
120 0.078
130 0.078
140 0.078
150 0.078
160 0.078
170 0.078
180 0.078
190 0.078
Depth 20
10 0.082
20 0.08
30 0.079
40 0.079
50 0.079
60 0.079
70 0.079
80 0.079
90 0.079
100 0.079
110 0.079
120 0.079
130 0.078
140 0.078
150 0.078
160 0.078
170 0.078
180 0.078
190 0.078
Depth 25
10 0.082
20 0.08
30 0.08
40 0.079
50 0.079
60 0.079
70 0.079
80 0.079
90 0.079
100 0.079
110 0.079
120 0.079
130 0.079
140 0.079
150 0.079
160 0.079
170 0.079
180 0.078
190 0.079


What's the best max_depth:

- 10
- **15**
- 20
- 25

Bonus question (not graded):

Will the answer be different if we change the seed for the model? **Maybe yes**

# Question 5

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorith, it finds the best split. When doint it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the imporatant features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

- Train the model with these parametes:
    - n_estimators=10,
    - max_depth=20,
    - random_state=1,
    - n_jobs=-1 (optional)
- Get the feature importance information from this model

In [45]:
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)
feature_names = [f'feature {i}' for i in range(df.shape[1])]
forest_importances = pd.Series(rf.feature_importances_, index=dv.get_feature_names()).sort_values()
forest_importances

neighbourhood_group=Staten Island    0.000141
neighbourhood_group=Bronx            0.000264
neighbourhood_group=Queens           0.001242
neighbourhood_group=Brooklyn         0.001477
room_type=Shared room                0.004774
room_type=Private room               0.006952
calculated_host_listings_count       0.031528
neighbourhood_group=Manhattan        0.039664
number_of_reviews                    0.045393
minimum_nights                       0.049416
reviews_per_month                    0.053890
availability_365                     0.072485
longitude                            0.146144
latitude                             0.146741
room_type=Entire home/apt            0.399888
dtype: float64

What's the most important feature?

- neighbourhood_group=Manhattan
- **room_type=Entire home/apt**
- longitude
- latitude

# Question 6

Now let's train an XGBoost model! For this question, we'll tune the eta parameter

- Install XGBoost
- Create DMatrix for train and validation
- Create a watchlist
- Train a model with these parameters for 100 rounds:

        xgb_params = {
            'eta': 0.3, 
            'max_depth': 6,
            'min_child_weight': 1,
            'objective': 'reg:squarederror',
            'nthread': 8,
            'seed': 1,
            'verbosity': 1,
        }

In [56]:
y_train

46147    1.660640
47470    1.595709
1905     1.819777
2952     1.673489
10617    1.685370
           ...   
2738     1.732356
25619    1.657941
40278    1.595709
17605    1.875710
22406    1.855302
Name: price, Length: 31292, dtype: float64

In [64]:
sc = MinMaxScaler(feature_range=(0,1))
sc_y_train = sc.fit_transform(np.expand_dims(y_train, axis=1))
sc_y_val = sc.transform(np.expand_dims(y_val, axis=1))

features = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=sc_y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=sc_y_val, feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'val')]

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
round(math.sqrt(mean_squared_error(sc_y_val, y_pred)),5)

0.03379

Now change eta first to 0.1 and then to 0.01

In [65]:
sc = MinMaxScaler(feature_range=(0,1))
sc_y_train = sc.fit_transform(np.expand_dims(y_train, axis=1))
sc_y_val = sc.transform(np.expand_dims(y_val, axis=1))

features = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=sc_y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=sc_y_val, feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'val')]

xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
round(math.sqrt(mean_squared_error(sc_y_val, y_pred)),5)

0.03365

In [66]:
sc = MinMaxScaler(feature_range=(0,1))
sc_y_train = sc.fit_transform(np.expand_dims(y_train, axis=1))
sc_y_val = sc.transform(np.expand_dims(y_val, axis=1))

features = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=sc_y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=sc_y_val, feature_names=features)
watchlist = [(dtrain, 'train'), (dval, 'val')]

xgb_params = {
    'eta': 0.01, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=100)
y_pred = model.predict(dval)
round(math.sqrt(mean_squared_error(sc_y_val, y_pred)),5)

0.09913

What's the best eta?

- 0.3
- **0.1**
- 0.01